In [1]:
from config import load_config
from model import Model

import torch.nn as nn
import torch
import torch.nn.functional as F

2024-04-26 15:23:48,011 - modelscope - INFO - PyTorch version 2.1.2 Found.
2024-04-26 15:23:48,014 - modelscope - INFO - Loading ast index from /home/xy/.cache/modelscope/ast_indexer
2024-04-26 15:23:48,048 - modelscope - INFO - Loading done! Current index file version is 1.10.0, with md5 6e478ed3c0dd6cfea723055784ffd6ea and a total number of 946 components indexed
/home/xy/.conda/envs/pytorch/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
model = Model("/data1/model/mistral-7b-base/")

Loading checkpoint shards: 100%|██████████| 2/2 [00:15<00:00,  7.53s/it]


In [3]:
logits, nlls = model.forward("Hello, world!")

In [4]:
logits, nlls, token_ids = model.forward("Hello, world!", return_tokens=True)
print(logits.shape, nlls.shape, token_ids.shape)

print(nlls)

torch.Size([5, 32000]) torch.Size([4]) torch.Size([1, 5])
tensor([6.7697, 1.4773, 5.7972, 0.3862])


In [5]:
logits, nlls, token_ids = model.forward("Hello, world!", return_tokens=True)

# Test if the above obtained nlls are equal to the ones obtained by the following code
criterian = nn.NLLLoss(reduction='none')
log_softmax = nn.LogSoftmax(dim=1)

logits = torch.unsqueeze(logits, 0)
# token_ids = torch.unsqueeze(token_ids, 0)
print(logits.shape, token_ids.shape)

torch.Size([1, 5, 32000]) torch.Size([1, 5])


In [6]:
logits = torch.permute(logits, (0, 2, 1)) # B,L,V -> B,V,L # Reason: nn.NLLLoss requires class to be of dimension 1
shift_logits = logits[:, :, :-1]
shift_token_ids = token_ids[..., 1:]

nlls2 = criterian(log_softmax(shift_logits), 
                  shift_token_ids)
print(nlls2)
print(nlls)
# Inconsistent

tensor([[6.7695, 1.4772, 5.7971, 0.3862]])
tensor([6.7697, 1.4773, 5.7972, 0.3862])


In [ ]:
# Try to fix nlls
logits, nlls, token_ids = model.forward("Hello, world!", return_tokens=True)

probs = F.softmax(logits, dim=-1) # shape: [L, V]
nlls_cands = torch.zeros(probs.shape[0], dtype=torch.float32)
for i in range(probs.shape[0]-1):
    nlls_cands[i] = -torch.log(probs[i, token_ids[0, i+1]])

print(nlls_cands) # Now, it is fixed